# Instalando bibliotecas

In [174]:
!pip install -r requirements.txt

In [175]:
!python -m spacy info
!python -m spacy info pt_core_news_lg


============================== Info about spaCy ==============================

spaCy version    3.7.5                         
Location         /usr/local/lib/python3.10/dist-packages/spacy
Platform         Linux-6.1.85+-x86_64-with-glibc2.35
Python version   3.10.12                       
Pipelines        pt_core_news_lg (3.7.0), en_core_web_sm (3.7.1)


=================== Info about pipeline 'pt_core_news_lg' ===================

lang                pt                            
name                core_news_lg                  
version             3.7.0                         
description         Portuguese pipeline optimized for CPU. Components: tok2vec, morphologizer, parser, lemmatizer (trainable_lemmatizer), senter, ner, attribute_ruler.
author              Explosion                     
email               contact@explosion.ai          
url                 https://explosion.ai          
license             CC BY-SA 4.0                  
spacy_version       >=3.7.0,<3.8.0  

In [176]:
import spacy
from spacy import displacy
from pymongo import MongoClient
import pandas as pd
import numpy as np
from spacy.language import Language
from spacy.tokenizer import Tokenizer
from collections import Counter
from spacy.matcher import Matcher
import os
from dotenv import load_dotenv, dotenv_values

In [177]:
load_dotenv()
# print(os.getenv("conn_str"))

True

# Conectando a base do MongoDB

In [178]:
conn_str = os.getenv("conn_str")

try:
    client = MongoClient(conn_str)
    print("Connected to MongoDB successfully!")
except Exception as e:
    print(f"Error: {e}")

Connected to MongoDB successfully!


# Acessando a base e coleção

In [179]:
db = client["Cluster"]
collection = db["Audios"]

db.list_collection_names()

['Audios']

# Identificando pelo nome do arquivo

In [180]:
query = {"file_name": '2968468.wav'}
projection = {"transcription": 1, "_id": 0}  # 1 inclui, 0 exclui

for transcricao in collection.find(query, projection):
  print(transcricao)

{'transcription': ' Boa tarde Liliane, meu nome é Viviane, eu falo aqui da TOTOS Matriz, tudo bem? Tudo bom Viviane. Eu gostaria por favor de falar com o Hélio. Ah sim, Viviane da TOTOS. TOTOS Matriz. Só mentindo. Tudo bem. Alô. Boa tarde, meu nome é Viviane, eu falo aqui da TOTOS Matriz, tudo bem? Tudo jóia. Eu falo com o Hélio? Isso. Hélio, eu estou entrando em contato com você para a gente fazer um rápido acompanhamento em relação a marca TOTOS que vocês utilizam hoje na empresa e no seu negócio. Hélio, eu estou aqui no departamento, você teria dois minutinhos para falar comigo? No momento eu estou bastante ocupado, seria só dois minutos mesmo? Sim, não, a pesquisa é bem rápida, só são algumas perguntas de zero a dez para a gente verificar como está a usabilidade e se vocês estão sendo bem atendidos. Tá, pode ser então. Certo, eu só vou fazer algumas confirmações por questões de segurança. O seu nome completo é Hélio Luiz Rodrigues e atualmente o senhor é um dos responsáveis pela ár

In [181]:
transcricao

{'transcription': ' Boa tarde Liliane, meu nome é Viviane, eu falo aqui da TOTOS Matriz, tudo bem? Tudo bom Viviane. Eu gostaria por favor de falar com o Hélio. Ah sim, Viviane da TOTOS. TOTOS Matriz. Só mentindo. Tudo bem. Alô. Boa tarde, meu nome é Viviane, eu falo aqui da TOTOS Matriz, tudo bem? Tudo jóia. Eu falo com o Hélio? Isso. Hélio, eu estou entrando em contato com você para a gente fazer um rápido acompanhamento em relação a marca TOTOS que vocês utilizam hoje na empresa e no seu negócio. Hélio, eu estou aqui no departamento, você teria dois minutinhos para falar comigo? No momento eu estou bastante ocupado, seria só dois minutos mesmo? Sim, não, a pesquisa é bem rápida, só são algumas perguntas de zero a dez para a gente verificar como está a usabilidade e se vocês estão sendo bem atendidos. Tá, pode ser então. Certo, eu só vou fazer algumas confirmações por questões de segurança. O seu nome completo é Hélio Luiz Rodrigues e atualmente o senhor é um dos responsáveis pela ár

# NLP Spacy

In [182]:
nlp = spacy.load("pt_core_news_lg")

doc = nlp(transcricao['transcription'])

* Identificando entidades

In [183]:
spacy.displacy.render(doc, style="ent", jupyter=True)

* Identificando possíveis notas

In [184]:
nlp = spacy.load("pt_core_news_lg")
entity_ruler = nlp.add_pipe("entity_ruler", before="ner")

patterns = [
    {
        "label": "NOTA",
        "pattern": [
            {
                'LOWER': {
                    'IN': ['zero', 'um', 'dois', 'três', 'quatro', 'cinco', 'seis', 'sete', 'oito', 'nove', 'dez',
                           '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
                }
            }
        ]
    }
]

entity_ruler.add_patterns(patterns)


doc = nlp(transcricao['transcription'])



colors = {"NOTA": "linear-gradient(315deg, #f5d020, #f53803)"}
options = {"ents": ["NOTA"], "colors": colors}


spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

In [185]:
doc = nlp(transcricao['transcription'])
type(doc)

spacy.tokens.doc.Doc

* Separar token/palavra

In [186]:
[token.text for token in doc]

[' ',
 'Boa',
 'tarde',
 'Liliane',
 ',',
 'meu',
 'nome',
 'é',
 'Viviane',
 ',',
 'eu',
 'falo',
 'aqui',
 'da',
 'TOTOS',
 'Matriz',
 ',',
 'tudo',
 'bem',
 '?',
 'Tudo',
 'bom',
 'Viviane',
 '.',
 'Eu',
 'gostaria',
 'por',
 'favor',
 'de',
 'falar',
 'com',
 'o',
 'Hélio',
 '.',
 'Ah',
 'sim',
 ',',
 'Viviane',
 'da',
 'TOTOS',
 '.',
 'TOTOS',
 'Matriz',
 '.',
 'Só',
 'mentindo',
 '.',
 'Tudo',
 'bem',
 '.',
 'Alô',
 '.',
 'Boa',
 'tarde',
 ',',
 'meu',
 'nome',
 'é',
 'Viviane',
 ',',
 'eu',
 'falo',
 'aqui',
 'da',
 'TOTOS',
 'Matriz',
 ',',
 'tudo',
 'bem',
 '?',
 'Tudo',
 'jóia',
 '.',
 'Eu',
 'falo',
 'com',
 'o',
 'Hélio',
 '?',
 'Isso',
 '.',
 'Hélio',
 ',',
 'eu',
 'estou',
 'entrando',
 'em',
 'contato',
 'com',
 'você',
 'para',
 'a',
 'gente',
 'fazer',
 'um',
 'rápido',
 'acompanhamento',
 'em',
 'relação',
 'a',
 'marca',
 'TOTOS',
 'que',
 'vocês',
 'utilizam',
 'hoje',
 'na',
 'empresa',
 'e',
 'no',
 'seu',
 'negócio',
 '.',
 'Hélio',
 ',',
 'eu',
 'estou',
 'aqui'

* Frequência de palavras

* Mais frequentes

In [187]:
complete_text = (transcricao['transcription'])
complete_doc = nlp(complete_text)

words = [
    token.text
    for token in complete_doc
    if not token.is_stop and not token.is_punct and not token.like_num
]

print(Counter(words).most_common(5))

[('Certo', 15), ('TOTOS', 9), ('contato', 8), ('Hélio', 6), ('gente', 6)]


* Menos frequentes

In [188]:
words = [
    token.text
    for token in complete_doc
    if not token.is_stop and not token.is_punct and not token.like_num
]

print(Counter(words).most_common()[:-6:-1])

[('pra', 1), ('Joia', 1), ('semana', 1), ('ótima', 1), ('Tenha', 1)]


* Demonstração de remoção de stop words

In [189]:
word_counter = Counter(
    [token.text for token in complete_doc if not token.is_punct]
)

# Print the 5 most common words
print(word_counter.most_common(5))

[('a', 35), ('o', 25), ('de', 19), ('Certo', 15), ('é', 13)]


In [190]:
def contar_palavras(texto):
    """
    Conta a quantidade total de palavras em um texto.

    Args:
        texto: O texto a ser analisado.

    Returns:
        O número total de palavras no texto.
    """
    palavras = texto.split()
    return len(palavras)

quantidade_palavras = contar_palavras(transcricao['transcription'])
print("O texto tem", quantidade_palavras, "palavras.")

O texto tem 694 palavras.


In [191]:
# Lista de palavras mágicas
palavras_magicas = ["obrigado", "obrigada", "por favor", "desculpa", "desculpe",
                     "por gentileza", "bom dia", "boa tarde", "boa noite",
                     "agradeço", "agradece", "gratidão", "sinto muito",
                     "perdão", "me perdoe", "com licença"]

# Conta a frequência de cada palavra mágica no texto
contagem_palavras_magicas = {palavra: (transcricao['transcription']).lower().count(palavra) for palavra in palavras_magicas}

# Calcula o número total de palavras mágicas
total_palavras_magicas = sum(contagem_palavras_magicas.values())

# Calcula o percentual de palavras mágicas no texto
percentual_palavras_magicas = (total_palavras_magicas / quantidade_palavras) * 100

# Imprime o percentual
print("Percentual de palavras mágicas no texto: {:.2f}%".format(percentual_palavras_magicas))

# Imprime a frequência de cada palavra mágica
for palavra, contagem in contagem_palavras_magicas.items():
    print("- {}: {} vezes".format(palavra, contagem))

Percentual de palavras mágicas no texto: 1.15%
- obrigado: 1 vezes
- obrigada: 3 vezes
- por favor: 2 vezes
- desculpa: 0 vezes
- desculpe: 0 vezes
- por gentileza: 0 vezes
- bom dia: 0 vezes
- boa tarde: 2 vezes
- boa noite: 0 vezes
- agradeço: 0 vezes
- agradece: 0 vezes
- gratidão: 0 vezes
- sinto muito: 0 vezes
- perdão: 0 vezes
- me perdoe: 0 vezes
- com licença: 0 vezes


* Separar quantidade de frases em um arquivo

In [192]:
about_doc = nlp(transcricao['transcription'])
sentences = list(about_doc.sents)
len(sentences)

87

In [193]:
for sentence in sentences:
  print(f"{sentence[:30]}...")

 ...
Boa tarde...
Liliane, meu nome é Viviane, eu falo aqui da TOTOS Matriz, tudo bem?...
Tudo bom Viviane....
Eu gostaria por favor de falar com o Hélio....
Ah sim, Viviane da TOTOS....
TOTOS Matriz....
Só mentindo....
Tudo bem....
Alô....
Boa tarde, meu nome é Viviane, eu falo aqui da TOTOS Matriz, tudo bem?...
Tudo jóia....
Eu falo com o Hélio?...
Isso....
Hélio, eu estou entrando em contato com você para a gente fazer um rápido acompanhamento em relação a marca TOTOS que vocês utilizam hoje na empresa e no seu...
Hélio, eu estou aqui no departamento, você teria dois minutinhos para falar comigo?...
No momento eu estou bastante ocupado, seria só dois minutos mesmo?...
Sim, não, a pesquisa é bem rápida, só são algumas perguntas de zero a dez para a gente verificar como está a usabilidade e se vocês estão...
Tá, pode ser então....
Certo, eu só vou fazer algumas confirmações por questões de segurança....
O seu nome completo é Hélio Luiz Rodrigues e atualmente o senhor é um dos responsá

* Delimitação

In [194]:
ellipsis_text = (transcricao['transcription'])

@Language.component("set_custom_boundaries")
def set_custom_boundaries(doc):
    """Add support to use `...` as a delimiter for sentence detection"""
    for token in doc[:-1]:
        if token.text == "...":
            doc[token.i + 1].is_sent_start = True
    return doc

In [195]:
custom_nlp = spacy.load("pt_core_news_lg")
custom_nlp.add_pipe("set_custom_boundaries", before="parser")
custom_ellipsis_doc = custom_nlp(ellipsis_text)
custom_ellipsis_sentences = list(custom_ellipsis_doc.sents)
for sentence in custom_ellipsis_sentences:
    print(sentence)

 
Boa tarde
Liliane, meu nome é Viviane, eu falo aqui da TOTOS Matriz, tudo bem?
Tudo bom Viviane.
Eu gostaria por favor de falar com o Hélio.
Ah sim, Viviane da TOTOS.
TOTOS Matriz.
Só mentindo.
Tudo bem.
Alô.
Boa tarde, meu nome é Viviane, eu falo aqui da TOTOS Matriz, tudo bem?
Tudo jóia.
Eu falo com o Hélio?
Isso.
Hélio, eu estou entrando em contato com você para a gente fazer um rápido acompanhamento em relação a marca TOTOS que vocês utilizam hoje na empresa e no seu negócio.
Hélio, eu estou aqui no departamento, você teria dois minutinhos para falar comigo?
No momento eu estou bastante ocupado, seria só dois minutos mesmo?
Sim, não, a pesquisa é bem rápida, só são algumas perguntas de zero a dez para a gente verificar como está a usabilidade e se vocês estão sendo bem atendidos.
Tá, pode ser então.
Certo, eu só vou fazer algumas confirmações por questões de segurança.
O seu nome completo é Hélio Luiz Rodrigues e atualmente o senhor é um dos responsáveis pela área...
É a área de 

* Posicionamento/Index do Token

In [196]:
about_text = (transcricao['transcription'])

about_doc = nlp(about_text)

for token in about_doc:
    print(token, token.idx)

  0
Boa 1
tarde 5
Liliane 11
, 18
meu 20
nome 24
é 29
Viviane 31
, 38
eu 40
falo 43
aqui 48
da 53
TOTOS 56
Matriz 62
, 68
tudo 70
bem 75
? 78
Tudo 80
bom 85
Viviane 89
. 96
Eu 98
gostaria 101
por 110
favor 114
de 120
falar 123
com 129
o 133
Hélio 135
. 140
Ah 142
sim 145
, 148
Viviane 150
da 158
TOTOS 161
. 166
TOTOS 168
Matriz 174
. 180
Só 182
mentindo 185
. 193
Tudo 195
bem 200
. 203
Alô 205
. 208
Boa 210
tarde 214
, 219
meu 221
nome 225
é 230
Viviane 232
, 239
eu 241
falo 244
aqui 249
da 254
TOTOS 257
Matriz 263
, 269
tudo 271
bem 276
? 279
Tudo 281
jóia 286
. 290
Eu 292
falo 295
com 300
o 304
Hélio 306
? 311
Isso 313
. 317
Hélio 319
, 324
eu 326
estou 329
entrando 335
em 344
contato 347
com 355
você 359
para 364
a 369
gente 371
fazer 377
um 383
rápido 386
acompanhamento 393
em 408
relação 411
a 419
marca 421
TOTOS 427
que 433
vocês 437
utilizam 443
hoje 452
na 457
empresa 460
e 468
no 470
seu 473
negócio 477
. 484
Hélio 486
, 491
eu 493
estou 496
aqui 502
no 507
departamento 510
, 

* Classificação da palavra

In [197]:
print(
    f"{'Text with Whitespace':22}"
    f"{'Is Alphanumeric?':15}"
    f"{'Is Punctuation?':18}"
    f"{'Is Stop Word?'}"
)

for token in about_doc:
    print(
        f"{str(token.text_with_ws):22}"
        f"{str(token.is_alpha):15}"
        f"{str(token.is_punct):18}"
        f"{str(token.is_stop)}"
    )

Text with Whitespace  Is Alphanumeric?Is Punctuation?   Is Stop Word?
                      False          False             False
Boa                   True           False             True
tarde                 True           False             True
Liliane               True           False             False
,                     False          True              False
meu                   True           False             True
nome                  True           False             False
é                     True           False             True
Viviane               True           False             False
,                     False          True              False
eu                    True           False             True
falo                  True           False             False
aqui                  True           False             True
da                    True           False             True
TOTOS                 True           False             False
Matriz                

* Removendo stop words
* Removendo pontuações


In [199]:
custom_about_text = (transcricao['transcription'])

nlp = spacy.load("pt_core_news_lg")
about_doc = nlp(custom_about_text)

print([token for token in about_doc if not token.is_stop and not token.is_punct])

[ , Liliane, nome, Viviane, falo, TOTOS, Matriz, Viviane, gostaria, falar, Hélio, Ah, Viviane, TOTOS, TOTOS, Matriz, mentindo, Alô, nome, Viviane, falo, TOTOS, Matriz, jóia, falo, Hélio, Hélio, entrando, contato, gente, rápido, acompanhamento, marca, TOTOS, utilizam, hoje, empresa, negócio, Hélio, departamento, teria, minutinhos, falar, comigo, ocupado, minutos, pesquisa, rápida, perguntas, gente, verificar, usabilidade, sendo, atendidos, Tá, Certo, vou, confirmações, questões, segurança, nome, completo, Hélio, Luiz, Rodrigues, atualmente, senhor, responsáveis, TI, empresa, né, exatamente, Certo, e-mail, continua, sendo, ti.casajoca.com.br, Certo, confirmações, Iniciando, avaliação, escala, probabilidade, senhor, recomendar, TOTOS, amigo, colega, hoje, Certo, gostaria, informar, motivo, nota, Certo, vou, perguntas, rápidas, informar, nota, precisa, falar, motivo, responder, caso, tenha, contato, Tá, jóia, agilidade, processos, suporte, nota, Certo, atendimento, agente, suporte, Certo, 

* Diferentes categorizações de palavras

In [200]:
lemma_text = (transcricao['transcription'])

lemma_doc = nlp(lemma_text)

for token in lemma_doc:
    if str(token) != str(token.lemma_):
        print(f"{str(token):>20} : {str(token.lemma_)}")

                   é : ser
                falo : falar
                  da : de o
                Tudo : tudo
                  Eu : eu
            gostaria : gostar
                  Ah : ah
                  da : de o
                  Só : só
            mentindo : mentir
                Tudo : tudo
                 Alô : alô
                   é : ser
                falo : falar
                  da : de o
                Tudo : tudo
                  Eu : eu
                falo : falar
                Isso : isso
               estou : estar
            entrando : entrar
                   a : o
                   a : o
               vocês : você
            utilizam : utilizar
                  na : em o
                  no : em o
               estou : estar
                  no : em o
               teria : ter
          minutinhos : minutinho
                  No : em o
               estou : estar
             ocupado : ocupar
               seria : ser
             min

* Demonstração de como uma palavra está sendo usada em uma frase

In [201]:
about_text = (transcricao['transcription'])

about_doc = nlp(about_text)

for token in about_doc:
    print(
        f"""
        TOKEN: {str(token)}
        =====
        TAG: {str(token.tag_):10} POS: {token.pos_}
        EXPLANATION: {spacy.explain(token.tag_)}"""
    )


        TOKEN:  
        =====
        TAG: SPACE      POS: SPACE
        EXPLANATION: space

        TOKEN: Boa
        =====
        TAG: ADJ        POS: ADJ
        EXPLANATION: adjective

        TOKEN: tarde
        =====
        TAG: ADV        POS: ADV
        EXPLANATION: adverb

        TOKEN: Liliane
        =====
        TAG: PROPN      POS: PROPN
        EXPLANATION: proper noun

        TOKEN: ,
        =====
        TAG: PUNCT      POS: PUNCT
        EXPLANATION: punctuation

        TOKEN: meu
        =====
        TAG: DET        POS: DET
        EXPLANATION: determiner

        TOKEN: nome
        =====
        TAG: NOUN       POS: NOUN
        EXPLANATION: noun

        TOKEN: é
        =====
        TAG: AUX        POS: AUX
        EXPLANATION: auxiliary

        TOKEN: Viviane
        =====
        TAG: PROPN      POS: PROPN
        EXPLANATION: proper noun

        TOKEN: ,
        =====
        TAG: PUNCT      POS: PUNCT
        EXPLANATION: punctuation

        

# Pré-processamento

In [202]:
complete_text = (transcricao['transcription'])

complete_doc = nlp(complete_text)

def is_token_allowed(token):
    return bool(
        token
        and str(token).strip()
        and not token.is_stop
        and not token.is_punct
    )

def preprocess_token(token):
    return token.lemma_.strip().lower()

complete_filtered_tokens = [
    preprocess_token(token)
    for token in complete_doc
    if is_token_allowed(token)
]

In [203]:
complete_filtered_tokens

['liliane',
 'nome',
 'viviane',
 'falar',
 'totos',
 'matriz',
 'viviane',
 'gostar',
 'falar',
 'hélio',
 'ah',
 'viviane',
 'totos',
 'totos',
 'matriz',
 'mentir',
 'alô',
 'nome',
 'viviane',
 'falar',
 'totos',
 'matriz',
 'jóia',
 'falar',
 'hélio',
 'hélio',
 'entrar',
 'contato',
 'gente',
 'rápido',
 'acompanhamento',
 'marca',
 'totos',
 'utilizar',
 'hoje',
 'empresa',
 'negócio',
 'hélio',
 'departamento',
 'ter',
 'minutinho',
 'falar',
 'comigo',
 'ocupar',
 'minuto',
 'pesquisa',
 'rápido',
 'pergunta',
 'gente',
 'verificar',
 'usabilidade',
 'ser',
 'atender',
 'tá',
 'certo',
 'ir',
 'confirmação',
 'questão',
 'segurança',
 'nome',
 'completo',
 'hélio',
 'luiz',
 'rodrigues',
 'atualmente',
 'senhor',
 'responsável',
 'ti',
 'empresa',
 'né',
 'exatamente',
 'certo',
 'e-mail',
 'continuar',
 'ser',
 'ti.casajoca.com.br',
 'certo',
 'confirmação',
 'iniciando',
 'avaliação',
 'escala',
 'probabilidade',
 'senhor',
 'recomendar',
 'totos',
 'amigo',
 'colega',
 'hoj

* Palavras linkadas e suas dependências

In [205]:
denp_text = transcricao['transcription']
denp_doc = nlp(denp_text)

for token in denp_doc:
    print(
        f"""
        TOKEN: {token.text}
        =====
        {token.tag_ = }
        {token.head.text = }
        {token.dep_ = }
        """
    )

A saída de streaming foi truncada nas últimas 5000 linhas.
        token.dep_ = 'cop'
        

        TOKEN: bastante
        =====
        token.tag_ = 'ADV'
        token.head.text = 'ocupado'
        token.dep_ = 'advmod'
        

        TOKEN: ocupado
        =====
        token.tag_ = 'ADJ'
        token.head.text = 'ocupado'
        token.dep_ = 'ROOT'
        

        TOKEN: ,
        =====
        token.tag_ = 'PUNCT'
        token.head.text = 'ocupado'
        token.dep_ = 'punct'
        

        TOKEN: seria
        =====
        token.tag_ = 'AUX'
        token.head.text = 'minutos'
        token.dep_ = 'cop'
        

        TOKEN: só
        =====
        token.tag_ = 'ADV'
        token.head.text = 'minutos'
        token.dep_ = 'advmod'
        

        TOKEN: dois
        =====
        token.tag_ = 'NUM'
        token.head.text = 'minutos'
        token.dep_ = 'nummod'
        

        TOKEN: minutos
        =====
        token.tag_ = 'NOUN'
        token.head

In [208]:
client.close()

In [207]:
# https://realpython.com/natural-language-processing-spacy-python/